In [13]:
import pymongo

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint

In [2]:
env_config = dotenv_values("../.env")

In [3]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")

In [14]:
mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

In [4]:
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

## Caso de uso 1 - Búsqueda de convocatorias de oposiciones a bibliotecario/as de las Cortes

Esto requiere de una búsqueda léxica con palabras clave.

In [26]:
query_params = {
    "match": {
        "text": "convocatoria oposiciones bibliotecario cortes"
    }
}
params = {
    "index": "boe",
    "query": query_params,
    "_source": ["doc_id", "text"]
}
results = es_client.search(**params)
matches = set(hit["_source"]["doc_id"] for hit in results["hits"]["hits"])

In [27]:
mongo_query = {"identificador": {"$in": list(matches)}}
mongo_result = mongo_collection.find(mongo_query)
for item in mongo_result:
    print(item["titulo"])

Resolución de 25 de abril de 2023, del Instituto Cervantes, por la que se publica el Convenio con Repsol Exploración Argelia, SA, para el desarrollo de actividades culturales y académicas del Instituto Cervantes en Argelia en 2023.
Resolución de 25 de abril de 2023, de la Universidad de Jaén, por la que se convoca proceso selectivo para la provisión, por el sistema general de acceso libre, de plaza de personal laboral en la categoría de Titulado/a de Grado Medio de Apoyo a la Docencia e Investigación.
Real Decreto 340/2023, de 3 de mayo, por el que se aprueba la provisión de plazas de las Fuerzas Armadas y de la Escala de Oficiales de la Guardia Civil para el año 2023.
Resolución de 25 de abril de 2023, de la Secretaría General del Tribunal Constitucional, por la que se convoca beca de formación en biblioteconomía y documentación relacionada con los fondos bibliográficos del Tribunal Constitucional.
Ley 5/2023, de 13 de abril, integral de medidas contra el despoblamiento y por la equid

## Caso de uso 2 - Búsqueda de entradas relacionadas con el control de inundaciones

In [24]:
query_text = "control de inundaciones"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
    "filter": {
        "match": {
            "text": "control de inundaciones",
        },
    },
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
results = es_client.search(**params)
matches = set(hit["_source"]["doc_id"] for hit in results["hits"]["hits"])

In [25]:
mongo_query = {"identificador": {"$in": list(matches)}}
mongo_result = mongo_collection.find(mongo_query)
for item in mongo_result:
    print(item["titulo"])

Resolución de 27 de abril de 2023, de la Confederación Hidrográfica del Guadalquivir, O.A., por la que se publica el Convenio con el Ayuntamiento de Jaén, para ejecución y financiación de las obras "Proyecto de intervención en el ámbito de los Cañones de Rio Frío en Jaén".
Resolución de 18 de abril de 2023, de la Dirección General de Política Energética y Minas, por la que se otorga a Green Capital Power, SL, autorización administrativa previa para la instalación fotovoltaica La Vega, de 109,26 MW de potencia instalada, y sus infraestructuras de evacuación asociadas, en Fuenlabrada, Humanes de Madrid, Parla, Griñón y Moraleja de Enmedio (Madrid).
Resolución de 18 de abril de 2023, de la Dirección General de Política Energética y Minas, por la que se otorga a Cobra Concesiones, SL, autorización administrativa previa para la Planta Solar Fotovoltaica "FV Castalla" con una potencia de 100 MW, y su infraestructura de evacuación, en Castalla (Alicante).
Resolución de 18 de abril de 2023, de